In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# this allows plots to appear directly in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
# pip install patsy
# pip install statsmodels
import statsmodels.formula.api as smf
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

from IPython.core.display import display, HTML

In [ ]:
# define url
url_0420 = '/Users/shaokuixing/Desktop/whatever/survey50%GenderAge20160420.csv'

# read csv file
rawData = pd.read_csv(url_0420, sep=';', thousands = '.', decimal =',')

In [ ]:
# subset recall
recall = rawData[rawData['questionid']==84].reset_index()
recall['recall'] = np.where(recall['ioptiontext']=='Yes',1,0).astype(int)
recall['recall_category'] = np.where(recall['ioptiontext']=='Yes','Yes','No')

# subset interest
interest = rawData[rawData['questionid']==86].reset_index()
interest['interest'] = pd.to_numeric(interest['ioptiontext'], errors='coerce')
interest['interest_category']=np.where(interest['interest']>= 4,'High','Low')

# subset intent
intent = rawData[rawData['questionid']==83].reset_index()
intent['intent'] = pd.to_numeric(intent['ioptiontext'], errors='coerce')
intent['intent_category']=np.where(intent['intent'] >= 4 ,'High','Low')

# subset age
age = rawData[rawData['questionid']==90].reset_index()
age=age.rename(columns = {'ioptiontext':'age_category'})

# subset gender
gender = rawData[rawData['questionid']==91].reset_index()
gender=gender.rename(columns = {'ioptiontext':'gender_category'})

In [ ]:
#intent[intent['intent_category'] == 0]
# df.sort(['c1','c2'], ascending=[False,True])
#interest.sort(['answersetid'], ascending=True).head()
#interest.sort_values(by="answersetid").sample(n=5, random_state=1)

In [ ]:
# drop unnecessary columns
recall.drop(recall[['ioptiontext','answerid','questionid','questiontext','optionid', 
                 'iscomplete','isrefgroup']],axis=1, inplace=True)
interest.drop(interest[['ioptiontext','answerid','questionid','questiontext','optionid', 
                 'iscomplete','isrefgroup']],axis=1, inplace=True)
intent.drop(intent[['ioptiontext','answerid','questionid','questiontext','optionid', 
              'iscomplete','isrefgroup']],axis=1, inplace=True)
age.drop(age[['answerid','questionid','questiontext','optionid','iscomplete','isrefgroup']],axis=1, inplace=True)
gender.drop(gender[['answerid','questionid','questiontext','optionid','iscomplete','isrefgroup']],axis=1, inplace=True)
#recall.head(1)
age.head()

In [ ]:
# merge data
df = recall.merge(interest, 
on= ['surveyid','answersetid',  'click', 
     'firstinteraction', 'deliveredimp', 'viewableimp','viewableimp_ic','totalviewtime']).merge(intent, on= ['surveyid','answersetid',  'click', 
     'firstinteraction', 'deliveredimp', 'viewableimp','viewableimp_ic','totalviewtime']).merge(age, on= ['surveyid','answersetid',  'click', 
     'firstinteraction', 'deliveredimp', 'viewableimp','viewableimp_ic','totalviewtime']).merge(gender, on= ['surveyid','answersetid',  'click', 
     'firstinteraction', 'deliveredimp', 'viewableimp','viewableimp_ic','totalviewtime'])
df.isnull().any()



In [ ]:
# create function to transform age column into numerical
def age_trans(x):
    if x['age_category'] == "15-24":
        val = 1
    elif x['age_category'] == "25-34":
        val = 2
    elif x['age_category'] == "35-44":
        val = 3
    elif x['age_category'] == "45-54":
        val = 4
    elif x['age_category'] == "55-64":
        val = 5
    else:
        val=6
    return val

# apply function and create numerical representation for age categories
df['age'] = df.apply(age_trans, axis=1)

In [ ]:
# transform gender into numerical column
df['gender'] = np.where(df['gender_category']=="Male", 1, 0)

In [ ]:
# reorder columns
df1= df[['answersetid', 'recall','interest','intent','age', 'gender','recall_category','interest_category', 
         'intent_category', 'age_category','gender_category',
         'click','firstinteraction','deliveredimp', 'viewableimp', 'viewableimp_ic','totalviewtime']]
df1.head()

In [ ]:
df1[df1['totalviewtime'] == 0].count()

In [ ]:
# calculate average view time
#df1.is_copy = False
#df1['avgviewtime'] = df1['totalviewtime'].div(df['viewableimp_ic'], axis=0)
df1.isnull().any()


In [ ]:
df2=df1.copy()
df2.is_copy = False
df2['avgviewtime'] = df2['totalviewtime'].div(df2['viewableimp_ic'], axis=0)
df2.isnull().any()


In [ ]:
# make sure no primary key is 0
np.isinf(df2.answersetid).any()

In [ ]:
# count how many rows are 0
df2[df2['totalviewtime']==0]['answersetid'].nunique()

In [ ]:
# count how many rows are null
# df.isnull().any().any()
df2.totalviewtime.isnull().values.sum()

In [ ]:
# let's look at all the null value rows
# df2[df2.isnull().any(axis=1)]
df2.head()

In [ ]:
df3 = df2.copy()
df3.head()

In [ ]:
df3['totalviewtime'].fillna(0, inplace=True)
df3['avgviewtime']=df3.avgviewtime.replace(np.inf, np.nan)
df3['avgviewtime'].fillna(0, inplace=True)
df3.head()

In [ ]:
df3[np.isinf(df3.avgviewtime)==True]

In [ ]:
# check how many inf value
np.isinf(df3.avgviewtime).any()

In [ ]:
# count how many rows are 0
df3[df3['avgviewtime']==0]['answersetid'].nunique()

In [ ]:
# check if any other value is converted to 0
df3[df3['answersetid']==0]['answersetid'].nunique()

In [ ]:
# count how many rows are null
# df.isnull().any().any()
df3.avgviewtime.isnull().values.sum()

In [ ]:
sns.heatmap(df1[df1.columns[1:]].corr(method='spearman'))

In [ ]:
#df3['avgviewtime'] = df3['avgviewtime']/1000
#df3['totalviewtime'] = df3['totalviewtime']/1000
df3[df3.columns[1:]].describe()

In [ ]:
df4=df3.copy()

In [ ]:
#categorical cut on avgviewtime
labels = [ "{0} - {1}".format(i, i + 2) for i in range(0, 120, 2) ]
print labels
df4['avgVTCategory'] = pd.cut(df4.avgviewtime, range(0, 121, 2), right=False, labels=labels)
df4[['avgviewtime','avgVTCategory']].sample(n=20, random_state=0)

In [ ]:
df2.head()

In [ ]:
df5 = df1.copy()
#df5['totalVT']=np.where(df5['viewableimp_ic']==0, 0, df5['totalviewtime'])
#df5.totalVT.isnull().values.sum()
df5.totalviewtime.head()

In [ ]:
import sqlite3
from pandas.io import sql
cnx = sqlite3.connect(':memory:', timeout=3000)
cnx.text_factory = str


In [ ]:
# write to database
sql.to_sql(df5, name = 'df5', con=cnx, if_exists='replace')

In [ ]:
# select data
sql.read_sql("select * from df5 limit 1",cnx)

In [ ]:
df5['totalVT']=sql.read_sql("select 1.00*(case when totalviewtime is null then 0 else totalviewtime end)/1000 as totalvt from df5",cnx)

In [ ]:
df6=df5.copy()

In [ ]:
df6['avgVT']=sql.read_sql("select case when totalVT = 0 or viewableimp_ic=0 then 0 else totalVT/viewableimp_ic end as avgVT from df5",cnx)

In [ ]:
df6.head(1)

In [ ]:
sql.to_sql(df6, name = 'df6', con=cnx, if_exists='replace')

In [ ]:
sql.read_sql("select sum(case when avgVT is null then 1 else 0 end) as numNUll from df6;",cnx)

In [ ]:
sql.read_sql("select *  from df6 where avgVT is null;",cnx)

In [ ]:
#categorical cut on avgviewtime
labels = [ "{0} - {1}".format(i, i + 2) for i in range(0, 120, 2) ]
print labels
df6['avgVTCategory'] = pd.cut(df6.avgVT, range(0, 121, 2), right=False, labels=labels)
df6[['avgVT','avgVTCategory']].sample(n=20, random_state=0)

In [ ]:
#categorical cut on totalviewtime
labels = [ "{0} - {1}".format(i, i + 5) for i in range(0, 1000, 5) ]
print labels
df6['totalVTCategory'] = pd.cut(df6.totalVT, range(0, 1001, 5), right=False, labels=labels)
df6[['totalVT','totalVTCategory']].sample(n=20, random_state=0)

In [ ]:
df6.head()

In [ ]:
from pivottablejs import pivot_ui
#pivot_ui(df6)

In [ ]:
# l = range(10)
# l[::2]         # even  - start at the beginning at take every second item
#[0, 2, 4, 6, 8]
# l[1::2]        # odd - start at second item and take every second item
#[1, 3, 5, 7, 9]
#categories = pd.cut(df['postTestScore'], bins, labels=group_names)
#df['categories'] = pd.cut(df['postTestScore'], bins, labels=group_names)

In [ ]:
def FuncAvgVTScale (x):
    if (x['avgVT'] >=0 ) & (x['avgVT']<=1) :  
        return '0-1'
    if (x['avgVT'] >1 ) & (x['avgVT']<=3) :  
        return '1-3'
    if (x['avgVT'] >3 ) & (x['avgVT']<=5) :  
        return '3-5'
    if (x['avgVT'] >5 ) & (x['avgVT']<=7) :  
        return '5-7'
    if (x['avgVT'] >7 ) & (x['avgVT']<=9) :  
        return '7-9'
    if (x['avgVT'] >9 ) & (x['avgVT']<=11) :  
        return '9-11'
    if (x['avgVT'] >11 ) & (x['avgVT']<=13) :  
        return '11-13'
    if (x['avgVT'] >13 ) & (x['avgVT']<=15) :  
        return '13-15'
    if (x['avgVT'] >15 ) & (x['avgVT']<=17) :  
        return '15-17'
    if (x['avgVT'] >17 ) & (x['avgVT']<=19) :  
        return '17-19'
    if (x['avgVT'] >19 ) & (x['avgVT']<=21) :  
        return '19-21'
    if (x['avgVT'] >21 ) & (x['avgVT']<=23) :  
        return '21-23'
    if (x['avgVT'] >23 ) & (x['avgVT']<=25) :  
        return '23-25'
    if (x['avgVT'] >25 ) & (x['avgVT']<=27) :  
        return '25-27'
    if (x['avgVT'] >27 ) & (x['avgVT']<=29) :  
        return '27-29'
    if (x['avgVT'] >29 ) & (x['avgVT']<=31) :  
        return '29-31'
    if (x['avgVT'] >31 ) & (x['avgVT']<=33) :  
        return '31-33'
    if (x['avgVT'] >33 ) & (x['avgVT']<=35) :  
        return '33-35'
    return 'above 35'

In [ ]:
df6['avgVTScale2']=df6.apply(lambda x: FuncAvgVTScale(x),axis=1)

In [ ]:
from pivottablejs import pivot_ui
#pivot_ui(df6)

In [ ]:
# cut data by percentile 
#qcdf = pd.DataFrame(range(1000))
#qcdf.columns = ['a1']
#qcdf['percentile'] = pd.qcut(qcdf.a1,10, labels=False)

df6['avgVTScale3'] = pd.qcut(df6.avgVT,10,labels=False)
df6.sample(n=10)

In [ ]:
from pivottablejs import pivot_ui
#pivot_ui(df6)
#sns.heatmap(df6.corr())
sns.heatmap(df6[df6.columns[1:]].corr(method='spearman'))

In [ ]:
df6[['avgVTScale3', 'answersetid']].corr(method='spearman')

In [ ]:
sns.lmplot(x='avgVTScale3', y='answersetid', data=df6, aspect=1.5,scatter_kws={'alpha':0.5} )
# the higher of answersetid the more recent the survey has been answered
# assumpiton 1: overall view time per impression has increased over the past 4 month period???
# assumption 2: the later an interview answers the campaign the more probable the person has seen it longer each time??

In [ ]:
df6.avgVT.quantile(0.6)

In [ ]:
scale3max=df6.groupby(['avgVTScale3'])['avgVT'].max()
scale3min=df6.groupby(['avgVTScale3'])['avgVT'].min()
scale3_range = pd.concat([scale3min, scale3max], axis=1).round(0)
scale3_range



In [ ]:
df6['interest_Category_Num']=sql.read_sql("select case when interest >= 4 then 10 else 0 end as interest_Category_Num from df5",cnx)


In [ ]:
df6['intent_Category_Num']=sql.read_sql("select case when intent >= 4 then 10 else 0 end as intent_Category_Num from df5",cnx)
df6.head(10)

In [ ]:
df6['totalVTScale3'] = pd.qcut(df6.totalVT,10,labels=False)
df6.sample(n=10)

In [ ]:
sns.heatmap(df6[['recall', 'interest_Category_Num', 'intent_Category_Num','age', 'gender','firstinteraction',
                 'viewableimp_ic', 'totalVTScale3','avgVTScale3']].corr())

In [ ]:
from pivottablejs import pivot_ui
pivot_ui(df6)

In [ ]:

df7 = df6[['recall', 'interest_Category_Num', 'intent_Category_Num','age', 'gender','firstinteraction','viewableimp_ic', 'totalVTScale3',
'avgVTScale3']][(df6.totalVTScale3 < df6.totalVTScale3.quantile(1)) & (df6.totalVTScale3 > df6.totalVTScale3.quantile(.30))].copy()

In [ ]:
sns.heatmap(df7.corr())

In [ ]:
df6.head(1)

In [ ]:
%reload_ext rpy2.ipython
%R help(package="MASS")
#%R install.packages("MESS",dependencies=TRUE)
#%R library(MESS)